# Alumno: Gerardo de Miguel González

##  Práctica Remuestreo

### Carga y limpieza de los datos

In [1]:
#::GMG::Cargamos los datos
data <- read.csv(file = 'm1965-stats-ii-datos-practica-remuestreo.csv',header = TRUE,sep = ',')

In [2]:
#::GMG::La columna de de fechas (date) está en un formato no compatible con el tipo de fecha
#       DD/MM/YYYY adecuado para una indexación posterior que vamos a necesitar para la selección
#       de los parámetros AVG.Cond. (Conductividad) y AVG.Salinity. (Salinidad) de 2014 ...
#       Ajustamos las fechas
data$date <- as.Date(x = data$date, format='%d/%m/%Y %H:%M')

In [3]:
#::GMG::Vamos a extraer los datos que necesitamos para el remuestreo
myvars <- names(data) %in% c("date", "AVG.Temp.")
data.processed <- data[myvars]
head(data.processed)

date,AVG.Temp.
2014-01-01,3.997746
2014-01-02,4.057667
2014-01-03,4.133612
2014-01-04,4.188685
2014-01-05,4.346172
2014-01-06,4.392661


In [4]:
#::GMG::Ahora vamos a separar los datos de "verano" de 2014 y 2015
# https://stackoverflow.com/questions/9407622/subsetting-a-dataframe-for-a-specified-month-and-year
data.processed.verano <- subset(
           x = data.processed, 
           #data.processed$date >= as.Date('2014-07-01') & 
           #data.processed$date <= as.Date('2014-09-31')
           format.Date(data.processed$date, '%m') >= '07' &
           format.Date(data.processed$date, '%m') <= '09'
           )
head(data.processed.verano)

,date,AVG.Temp.
182,2014-07-01,15.41396
183,2014-07-02,14.70326
184,2014-07-03,14.76020
185,2014-07-04,15.12685
186,2014-07-05,15.19523
187,2014-07-06,15.24690


In [5]:
#::GMG::Ahora selecciono y limpio el vector de temperatura de 2014 y 2015
#       Hay que limpiar los NaNs
#
x.verano <- na.omit(data.processed.verano$AVG.Temp.)
length(x.verano)

[1] 182

### Media y desviación muestral

**::GMG::** Cálculo de la media y deviación estándar muestral

In [7]:
#::GMG::Usamos las funciones que nos proporciona R
media.x.verano <- mean(x.verano)
sd.x.verano <- sd(x.verano)
paste('Media:',round(media.x.verano,3),' - Desv. Estándar:', round(sd.x.verano,3))

[1] "Media: 18.066  - Desv. Estándar: 1.971"

### Bootstrap

Crea una función que reciba un vector de números ‘x’ (la muestra original) y un número
natural N, que genere una matriz que tenga N columnas siendo cada una de estas, una de las
muestras de bootstrap.

In [20]:
b.mean <- function(x, N) {
    resamples <- lapply(1:N, function(i) sample(x, replace=T))
    #r.mean <- sapply(resamples, mean)
    #std.err <- sqrt(var(r.mean))
    #list(std.err=std.err, resamples=resamples, means=r.mean)
    return (as.matrix(resamples))
}

In [21]:
b <- b.mean(x.verano,10)

In [22]:
dim(b)

[1] 10  1